In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_signals
import gta_statistics
import gta_trades
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System1 look back period.
p2 = 24 # System2 look back period.
p3 = 12 # System3 look back period.
p4 = 6  # System4 look back period.
strategy = f'Donchian'
position_size = 7500  # Position size in major currency unit.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
# Function definitions.
def shares(close, position_size):
    '''Calculate number of shares for a given position size.'''
    shares = int(position_size / close)
    while shares % 4 != 0:
        shares += 1
    return shares

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm, history_start)

In [5]:
# Donchian channels.
dc1 = gta_indicator.donchian(prices, p1)
dc2 = gta_indicator.donchian(prices, p2)
dc3 = gta_indicator.donchian(prices, p3)
dc4 = gta_indicator.donchian(prices, p4)

In [6]:
# System1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = gta_signals.state_signal(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)
t1 = pd.concat([s1.entry, s1.exit, s1.close], axis=1)
t1 = t1[(t1.entry == 1) | (t1.exit == 1)]

In [7]:
# System2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = gta_signals.state_signal(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)
t2 = pd.concat([s2.entry, s2.exit, s2.close], axis=1)
t2 = t2[(t2.entry == 1) | (t2.exit == 1)]

In [8]:
# System3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = gta_signals.state_signal(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)
t3 = pd.concat([s3.entry, s3.exit, s3.close], axis=1)
t3 = t3[(t3.entry == 1) | (t3.exit == 1)]

In [9]:
# System4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = gta_signals.state_signal(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)
t4 = pd.concat([s4.entry, s4.exit, s4.close], axis=1)
t4 = t4[(t4.entry == 1) | (t4.exit == 1)]

In [10]:
# Trade list
td = pd.concat([t1, t2, t3, t4], axis=0)
td = td.sort_index()
td = td.drop_duplicates()
td

,entry,exit,close
date,,,
2011-08-26,1,0,2.260
2011-10-14,0,1,2.325
2012-06-01,0,1,3.450
2013-03-29,0,1,3.625
2013-08-02,0,1,3.510
2016-05-13,1,0,0.420
2016-06-24,0,1,0.440
2017-02-17,0,1,0.820
2017-03-10,0,1,0.775


In [15]:
shares = pd.DataFrame()
shares = pd.concat([td.entry, td.close], axis=1)
shares['raw'] = np.where(td.entry == 1, (position_size / td.close).astype('int'), 0)
# for i in range(len(shares)):
#     while shares[i] % 4 != 0:
#         shares[i] += 1
shares
# td['shares'] = shares
# td

,entry,close,raw
date,,,
2011-08-26,1,2.260,3318
2011-10-14,0,2.325,0
2012-06-01,0,3.450,0
2013-03-29,0,3.625,0
2013-08-02,0,3.510,0
2016-05-13,1,0.420,17857
2016-06-24,0,0.440,0
2017-02-17,0,0.820,0
2017-03-10,0,0.775,0
